# Monthly wages for females and males
In LOCAL CURRENCY UNITS

In [93]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import json
import csv
from collections import defaultdict
import functools
import operator

In [94]:
# Read csv files from No Ceilings project folder
path = "/Users/ericaxia/Downloads/Github/project-girlboss/data/noceilings-data-master/csv"
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
# Narrow down to just the files I want
files = ['MONWAGFE.csv', 'MONWAGMA.csv']
dfs = []
for f in files:
    df = pd.read_csv(f)
    dfs.append((f, df))    

## Let's narrow down to a chosen year

In [95]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")

In [96]:
# import US specific data (median dollar earnings per month)
us_data = pd.read_csv("median_US_earnings.csv")
us_data.head()  

,year,female,male
0,1995,3133,4386
1,1996,3216,4360
2,1997,3316,4471
3,1998,3388,4630
4,1999,3376,4668


In [97]:
wages_fem = dfs[0][1]  # monthly wages for females
wages_m = dfs[1][1] # monthly wages for males


In [98]:
wages_fem = wages_fem.iloc[0:85, :]  # rm bottom descrip rows
wages_m = wages_m.iloc[0:85, :]  # rm bottom descrip rows

In [99]:
# narrow down to countries that DO have data for 2011
wages_fem = wages_fem[~wages_fem['2011'].isna()]
wages_m = wages_m[~wages_m['2011'].isna()]
print(wages_fem.shape, wages_m.shape) # (26,18) -> we have 26 countries

(26, 18) (26, 18)


In [100]:
wages_fem = wages_fem.fillna(axis=0, method='pad')
wages_m = wages_m.fillna(axis=0, method='pad')

In [101]:
print(wages_fem.isna().sum().sum())
print(wages_m.isna().sum().sum())

0
0


In [102]:
# process us data separartely
us_data2 = us_data.transpose()
us_data2.columns = us_data2.iloc[0,:]
# us_data2['ISO'] = 'USA'
wages_fem_us = pd.DataFrame(us_data2.iloc[1,:])
wages_m_us = pd.DataFrame(us_data2.iloc[2,:])


In [103]:
wages_m_us.rename(columns={'male': 'value'}, inplace=True)
wages_fem_us.rename(columns={'female': 'value'}, inplace=True)


In [104]:
wages_fem_us['gender'] = 'female'
wages_fem_us.reset_index(inplace=True)
wages_m_us['gender'] = 'male'
wages_m_us.reset_index(inplace=True)

In [105]:
wages_us = pd.concat([wages_fem_us, wages_m_us], axis=0)
wages_us['ISO'] = 'USA'
wages_us2 = wages_us[wages_us['year'] != 'ISO']
# wages_us2.drop(labels='index', axis=1, inplace=True)

In [106]:
wages_us2.head()

,year,value,gender,ISO
0,1995,3133,female,USA
1,1996,3216,female,USA
2,1997,3316,female,USA
3,1998,3388,female,USA
4,1999,3376,female,USA


In [107]:
## combine BOTH genders one dataset for ONE country
""" 
1. Narrow down country
2. Separately for each M / F dataset, melt the dataset
3. concat the melted datasets together
"""
all_countries = pd.DataFrame()

## Choose a country
for c in wages_fem.ISO.unique():
# c = 'AUS'
    f1 = wages_fem[wages_fem['ISO'] == c]
    f2 = f1.melt().iloc[1:, :]
    f2.columns = ['year', 'value']
    f2['gender'] = 'female'
    m1 = wages_m[wages_m['ISO'] == c]
    m2 = m1.melt().iloc[1:, :]
    m2.columns = ['year', 'value']
    m2['gender'] = 'male'
    all = pd.concat([f2, m2], axis=0)
    all2 = all[all['year'] != 'gender']
    all2['ISO'] = c
    # all2.head()
    all_countries = pd.concat([all_countries, all2], axis=0)


In [108]:
# all_countries.head()
print(all_countries.shape)

(884, 4)


In [109]:
# all_countries.reset_index(inplace=True)
# wages_us2.reset_index(inplace=True)
all_countries2 = pd.concat([all_countries, wages_us2], axis=0)
print(all_countries2.shape)

(938, 4)


In [110]:
# all_countries2.head()
# all_countries2.tail()

In [111]:
# df = all_countries2.loc[:,~all_countries2.columns.duplicated()]

In [112]:
## check for missing values
# df.isna().sum().sort_values(ascending=False)
# df.drop(labels='index', axis=1, inplace=True)

In [113]:
## To Map ISO -> Country name for better understandability in graph
with open('iso_to_country_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    country_names = {rows[0]: rows[1] for rows in reader}
    # print(country_names)

country_names_dict = { v: k for k, v in country_names.items()}

def convert_iso_to_name(iso):
    if iso in country_names_dict:
        return country_names_dict[iso]
    else:
        return iso

In [114]:
df = all_countries2

In [115]:
df['country'] = df['ISO']
# df['country'] = df['ISO'].apply(convert_iso_to_name)
df.drop(labels='ISO', axis=1, inplace=True)

In [116]:
df['value'] = df['value'].astype(int)

In [117]:
df

,year,value,gender,country
1,1995,1867,female,AUS
2,1996,1923,female,AUS
3,1997,1996,female,AUS
4,1998,2043,female,AUS
5,1999,2081,female,AUS
...,...,...,...,...
22,2016,4585,male,USA
23,2017,4532,male,USA
24,2017,4536,male,USA
25,2018,4691,male,USA


In [118]:
df = df[['year', 'country', 'value', 'gender']]

In [119]:

df.to_csv("wages.csv", index=False)